In [1]:
import csv
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup

url_recetas = "https://medlineplus.gov/spanish/recetas/desayuno/"

In [2]:
response = requests.get(url_recetas)
enlaces_secciones = []

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    enlaces_secciones.append("https://medlineplus.gov/spanish/recetas/desayuno/")
    div_mp_content = soup.find('div', class_='mp-content')
    if div_mp_content:
        sections = div_mp_content.find_all('section')
        if len(sections) > 1:
            segunda_section = sections[1]
            links = segunda_section.find_all('a')
            for link in links:
                href = link.get('href')
                if href:
                    enlaces_secciones.append(href)
        else:
            print("No se encontró una segunda sección.")
    else:
        print("No se encontró el div con la clase mp-content.")

else:
    print(f"Error al acceder a la página: {response.status_code}")

print(enlaces_secciones)

['https://medlineplus.gov/spanish/recetas/desayuno/', 'https://medlineplus.gov/spanish/recetas/almuerzo/', 'https://medlineplus.gov/spanish/recetas/cena/', 'https://medlineplus.gov/spanish/recetas/postres/', 'https://medlineplus.gov/spanish/recetas/acompanamientos/', 'https://medlineplus.gov/spanish/recetas/bajo-en-grasa/', 'https://medlineplus.gov/spanish/recetas/bajo-en-sodio/', 'https://medlineplus.gov/spanish/recetas/bebidas/', 'https://medlineplus.gov/spanish/recetas/bocadillos/', 'https://medlineplus.gov/spanish/recetas/ensaladas/', 'https://medlineplus.gov/spanish/recetas/pan/', 'https://medlineplus.gov/spanish/recetas/salsas-y-aderezos/', 'https://medlineplus.gov/spanish/recetas/sin-gluten/', 'https://medlineplus.gov/spanish/recetas/sin-lacteos/', 'https://medlineplus.gov/spanish/recetas/sopas/', 'https://medlineplus.gov/spanish/recetas/vegetariana/']


In [3]:
mat_enlaces = []

categorias = [
    "Desayuno",
    "Almuerzo",
    "Cena",
    "Postres",
    "Acompañamientos",
    "Bajo en grasa",
    "Bajo en sodio",
    "Bebidas",
    "Bocadillos",
    "Ensaladas",
    "Pan",
    "Salsas y aderezos",
    "Sin gluten",
    "Sin lácteos",
    "Sopas",
    "Vegetariana"
]

diccionario_enlaces = {categoria: [] for categoria in categorias}

i = 0

for enlace in enlaces_secciones:

    response = requests.get(enlace)
    soup = BeautifulSoup(response.content, "html.parser")

    div_cards = soup.find_all('div', class_='recipe-card-block-with-height')


    for card in div_cards:
        diccionario_enlaces[categorias[i]].append(card.find("a").get("href"))

    i = i + 1


In [4]:
# max_len = max(len(v) for v in diccionario_enlaces.values())

# for clave, lista in diccionario_enlaces.items():
#     if len(lista) < max_len:
#         diccionario_enlaces[clave] += [None] * (max_len - len(lista))

# df = pd.DataFrame(diccionario_enlaces)

# df.head(100)

In [6]:
def extraer_numeros(cadena):
  numeros = ""
  for caracter in cadena:
    if caracter.isdigit():
      numeros += caracter
  return numeros

def extraer_nombre_vit_min(cadena):
  """Extrae el nombre de la vitamina de una cadena."""
  partes = cadena.split()
  return " ".join(partes[:-1])

vit_min = set()

# for categoria, lista in diccionario_enlaces.items():

#     for enlace in lista: # Reemplazar por categoría
       
#         response = requests.get(enlace)

#         if response.status_code == 200:
#             soup = BeautifulSoup(response.content, "html.parser")

#             facts_nutri = soup.find('div', class_='facts-nutri').find_all("span", class_="fact-left")

#             for span in facts_nutri:
#                 aux = extraer_nombre_vit_min(span.text)
#                 vit_min.add(aux)

# print(vit_min)

In [7]:
 # "Categoría";"Enlace";"Titulo receta";"imagen";"Ingredientes";"Calorías";"Grasa Total";"Colesterol";"Sodio";"Carbohidrato Total";"Proteina";"Vitamina A";"Vitamina C";"Vitamina D";"Calcio";"Hierro";"Potasio";

datos = []

datos.append(["Categoria","Enlace","Nombre","ImagenURL","Ingredientes","Calorias","Grasa Total(gr)","Colesterol(mg)","Sodio(mg)","Carbohidrato Total(gr)","Proteina(gr)","Vitamina A(mcg)","Vitamina C(mg)","Vitamina D(mcg)","Calcio(mg)","Hierro(mg)","Potasio(mg)"])

for categoria, lista in diccionario_enlaces.items():
    print(categoria)

    for enlace in lista: # Reemplazar por categoría
        
        response = requests.get(enlace)

        ingredientes = ""
        factsM = ""
        factsN = ""

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            # SCRAPPING NOMBRE
            
            nombre = soup.find('h1').text

            # SCRAPPING IMAGEN

            imagen = soup.find('div', class_='mp-img').find("img").get("src")

            # SCRAPPING CALORÍAS
            
            calorias = soup.find_all('span', class_='fact-right')[1].find("strong").text
            
            # SCRAPPING INGREDIENTES

            lista_ingredientes = soup.find('ul', class_='bulletlist')

            if lista_ingredientes:
                lista_ingredientes = lista_ingredientes.find_all("li")

                for li in lista_ingredientes:
                    ingredientes = ingredientes + " | " + li.text.strip()
            else :
                ingredientes = ""

            # SCRAPPING FACTS MAIN

            facts_main = soup.find('div', class_='facts-main').find_all("span", class_="fact-left")

            grasa_total = 0
            colesterol = 0
            sodio = 0
            carbohidrato_total = 0
            proteina = 0

            for span in facts_main:
                #factsM = factsM + " | " + span.text.strip()

                if "Grasa To" in span.text:
                    grasa_total = extraer_numeros(span.text)

                elif "Colesterol" in span.text:

                    colesterol = extraer_numeros(span.text)
                    
                elif "Sodio" in span.text:

                    sodio = extraer_numeros(span.text)

                elif "Carbohidrato Total" in span.text:

                    carbohidrato_total = extraer_numeros(span.text)

                elif "Prote" in span.text:
                    proteina = extraer_numeros(span.text)
                    
            # SCRAPPING FACTS NUTRI

            facts_nutri = soup.find('div', class_='facts-nutri').find_all("span", class_="fact-left")

            vitamina_a = 0
            vitamina_c = 0
            vitamina_d = 0
            calcio = 0
            hierro = 0
            potasio = 0

            for span in facts_nutri:
        # "Categoría";"Enlace";"Titulo receta";"imagen";"Ingredientes";"Calorías";"Grasa Total";"Colesterol";"Sodio";"Carbohidrato Total";"Proteina";"Vitamina A";"Vitamina C";"Vitamina D";"Calcio";"Hierro";"Potasio";
                
                if "Vitamina A" in span.text:
                    vitamina_a = extraer_numeros(span.text)

                elif "Vitamina C" in span.text:

                    vitamina_c= extraer_numeros(span.text)
                    
                elif "Vitamina D" in span.text:

                    vitamina_d = extraer_numeros(span.text)

                elif "Calcio" in span.text:

                    calcio = extraer_numeros(span.text)

                elif "Hierro" in span.text:
                    hierro = extraer_numeros(span.text)

                elif "Potasio" in span.text:
                    potasio = extraer_numeros(span.text)
                else:
                    print("Tipo de dato no encontrado")

        else:
            print(f"Error al acceder a la página: {response.status_code}")

        datos.append([categoria,enlace,nombre,imagen,ingredientes,calorias,grasa_total,colesterol,sodio,carbohidrato_total,proteina,vitamina_a,vitamina_c,vitamina_d,calcio,hierro,potasio,])
        
    os.system('cls')

with open("assets/data/datos.csv", "w", newline="", encoding="utf-8") as archivo_csv:
    escritor = csv.writer(archivo_csv, delimiter=";")

    # Escribir la cabecera (opcional)
    escritor.writerow(datos[0])

    # Escribir los datos
    escritor.writerows(datos[1:])

 # "Categoría";"Enlace";"Titulo receta";"imagen";"Ingredientes";"Calorías";"Grasa Total";"Colesterol";"Sodio";"Carbohidrato Total";"Proteina";"Vitamina A";"Vitamina C";"Vitamina D";"Calcio";"Hierro";"Potasio";

Desayuno
Almuerzo
Cena
Postres
Acompañamientos
Bajo en grasa
Bajo en sodio
Bebidas
Bocadillos
Ensaladas
Pan
Salsas y aderezos
Sin gluten
Sin lácteos
Sopas
Vegetariana
